# **Inscribed square problem with Xpress NonLinear and Xpress Global**

***inscribed_square.ipynb***

This example shows how to use FICO&reg; Xpress NonLinear (SLP) or FICO&reg; Xpress Global to solve an instance of the inscribed square problem.

<!--*This example requires a license for the FICO&reg; Xpress Global solver. Click on [this link](https://www.fico.com/en/fico-xpress-trial-and-licensing-options) for more information about trial and licensing options.*-->

&copy; Copyright 2025 Fair Isaac Corporation

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.
 
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

This example uses FICO&reg; Xpress software. By running it, you agree to the Community License terms of the [Xpress Shrinkwrap License Agreement](https://community.fico.com/s/contentdocument/06980000002h0i5AAA) with respect to the FICO&reg; Xpress software. See the [licensing options](https://www.fico.com/en/fico-xpress-trial-and-licensing-options) overview for additional details and information about obtaining a paid license.

In [ ]:
# Install the xpress package
%pip install -q xpress

The inscribed square problem, also known as the square peg problem or the Toeplitz' conjecture, is an unsolved question in geometry: Does every plane simple closed curve contain all four vertices of some square? The problem was proposed by [Otto Toeplitz in 1911](https://en.wikipedia.org/wiki/Inscribed_square_problem).
The next example is a special instance of the problem, and computes a **maximal inscribing square** for the curve defined by: 

$$(\sin(t) \cdot \cos(t), \sin(t) \cdot t), t \in [-\pi,\pi]$$

[Source in MINLPLIB](https://www.minlplib.org/inscribedsquare01.html)

The problem is formulated using decision variables $t_i, \forall i = 1..4$, with values in the range $[-\pi,\pi]$, corresponding to the four values of the parameter $t$. Then, $x$ and $y$ are the coordinates of the first corner of the square, while ($a, b$) is a vector pointing to a second vertex. The remaining vertices are given by combining $x, y, a, b$. The length of the vector ($a, b$) is exactly the side length of the square, which we aim at maximizing.

The first two constraints define $x$ and $y$ as the coordinates of the first point:
$$
\begin{array}{llll}
& \qquad \sin(t_1) \cdot \cos(t_1) = x \\
& \qquad \sin(t_1) \cdot t_1 = y \\
\end{array}
$$

The next two constraints define the coordinates of the second vertex by adding the pointed vector ($a$, $b$) to the first point ($x$, $y$):
$$
\begin{array}{llll}
& \qquad \sin(t_2) \cdot \cos(t_2) = x + a\\
& \qquad \sin(t_2) \cdot t_2 = y + b \\
\end{array}
$$

The remaining four constraints make sure that we define a square by defining its remaining two vertices as combinations of ($x$, $y$) and the vector ($a$, $b$):
$$
\begin{array}{llll}
& \qquad \sin(t_3) \cdot \cos(t_3) = x - b \\
& \qquad \sin(t_3) \cdot t_3 = y + a \\
& \qquad \sin(t_4) \cdot \cos(t_4) = x + a - b \\
& \qquad \sin(t_4) \cdot t_4 = y + a + b \\
\end{array}
$$

The objective is to maximize the length of one (or each) side of the square:
$$
\begin{array}{llll}
& \qquad \max a^2 + b^2
\end{array}
$$

In [ ]:
import xpress as xp
import math
import numpy as np
from matplotlib import pyplot as plt
from collections import namedtuple

p = xp.problem()

# Add the variables, note that we force the (a, b) vector to have positive components to break symmetry.
t1 = p.addVariable(name="t1", lb=-math.pi, ub=math.pi) # t1..t4 are the four values of the parameter t.
t2 = p.addVariable(name="t2", lb=-math.pi, ub=math.pi)
t3 = p.addVariable(name="t3", lb=-math.pi, ub=math.pi)
t4 = p.addVariable(name="t4", lb=-math.pi, ub=math.pi)
x = p.addVariable(name="x")                            #  x and y are the (x,y) coordinates of the first corner of the square.
y = p.addVariable(name="y")
a = p.addVariable(name="a", lb=0)                      # (a, b) is a vector pointing to a second vertex, all the other vertices are given by a combination of x..b.
b = p.addVariable(name="b", lb=0)                    

# set initial values for the local solvers
p.nlpsetinitval([t1, t2, t4, x, y, a, b], [-math.pi, -math.pi/2, math.pi/2, 0, 0, 1, 1])

# the first two constraints define x and y as the coordinates of the first point
p.addConstraint(xp.sin(t1) * xp.cos(t1) == x)
p.addConstraint(xp.sin(t1) * t1         == y)
# the next two constraints define the coordinates of the second vertex by adding the pointed vector (a,b) to the first point (x,y).
p.addConstraint(xp.sin(t2) * xp.cos(t2) == x + a)
p.addConstraint(xp.sin(t2) * t2         == y + b)
# the remaining four constraints make sure that we define a square by defining its remaining two vertices as combinations of (x,y) and the vector (a,b)
p.addConstraint(xp.sin(t3) * xp.cos(t3) == x - b)
p.addConstraint(xp.sin(t3) * t3         == y + a)
p.addConstraint(xp.sin(t4) * xp.cos(t4) == x + a - b)
p.addConstraint(xp.sin(t4) * t4         == y + a + b)

# the objective is to maximize the length of one (or each) side of the square
p.setObjective(a**2 + b**2,sense=xp.maximize)

# choose between solving with a local or global solver
p.controls.nlpsolver = 1                       # 1: local, 2: global, -1: solver decides based on license and user functions / multistart jobs (default)
# choose between solving with SLP or Knitro
p.controls.localsolver = 0                     # 0: SLP, 1: Knitro, 2: use Optimizer (for convex QPs only), -1: automatic selection based on model characteristics and solver availability (default)

p.optimize()


The specific nonlinear solver can be chosen using the [NLPSOLVER](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/nonlinear/HTML/XSLP_NLPSOLVER.html) control:
  - Setting a value of "1" will invoke a local solver, with it being either SLP or Knitro (**if a license is available. The community license does not include Knitro**). 
  - The control defaults to -1, which has Xpress choose the most appropriate solver. If there are user functions or multi-start jobs, or if there is no license available for Xpress Global, a local solver will be called. In all other cases, Xpress Global will be used to solve the problem.

If a local solve is invoked, the [LOCALSOLVER](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/nonlinear/HTML/XSLP_SOLVER.html) control specify if either SLP or Knitro should be used by setting the control to "1" or "2" respectively. For convex QPs, setting the value to "2" will set Xpress Optimizer to solve the problem. By default (-1), an automatic selection is made based on model characteristics and solver availability.

By running the previous code cell below with the NLPSOLVER control equal to 1, SLP or Knitro will be called and the problem is solved to local optimality. This can be confirmed by running the code cell again with NLPSOLVER equal to 2, which finds a larger (maximal) square.


In [ ]:
# Visualize solution
Point = namedtuple('Point', 'x y')

def get_coords(t):

  x = np.sin(t) * np.cos(t)
  y = np.sin(t) * t
  return Point(x, y)

# below code approximates the graph of the function
t = np.arange(-math.pi, math.pi, 0.01)
xs = np.sin(t) * np.cos(t)
ys = np.sin(t) * t

# compute the x- and y-coordinates of the four vertices of the square
v1 = get_coords(p.getSolution(t1))
v2 = get_coords(p.getSolution(t2))
v3 = get_coords(p.getSolution(t3))
v4 = get_coords(p.getSolution(t4))

# plot the graph and the square
fig, ax = plt.subplots()
cells = ax.plot(xs, ys)
plt.plot([v1.x, v2.x], [v1.y, v2.y], 'ro-')
plt.plot([v2.x, v4.x], [v2.y, v4.y], 'ro-')
plt.plot([v4.x, v3.x], [v4.y, v3.y], 'ro-')
plt.plot([v3.x, v1.x], [v3.y, v1.y], 'ro-')

ax.set_aspect('equal', adjustable='box')

plt.show()

#print the variable values
print('solution: t1:', p.getSolution(t1), ', t2:', p.getSolution(t2), ', t3:', p.getSolution(t3), ', t4:', p.getSolution(t4), ', x:', p.getSolution(x), ', y:', p.getSolution(y), ', a:', p.getSolution(a), ', b:', p.getSolution(b))